In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer, load_digits, load_iris, make_blobs

## Classification vs. Regression

- **Regression**: Problem setting where we are predicting a *continuous* target
- **Classification**: Problem setting where we are predicting a *categorical* target

In [ ]:
#synthetic dataset
X, y = make_blobs( centers = 2, random_state=22)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c = y)
plt.grid()
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Synthetic Classification Data');

**PROBLEM**: Can you determine an equation of a line that separates the purple class from the yellow class?

In [ ]:
#answer in slack -- equation of line


### Our Motivating Example



In [ ]:
default = pd.read_csv('https://raw.githubusercontent.com/jfkoehler/nyu_bootcamp_fa24/refs/heads/main/data/Default.csv', index_col = 0)

In [ ]:
default.info()

In [ ]:
default.head(2)

### Visualizing Default by Continuous Features

In [ ]:
#scatterplot of balance vs. income colored by default status

In [ ]:
sns.scatterplot(data = default, x = 'balance', y = 'income', hue = 'default')

In [ ]:
#boxplots for balance and income by default
fig, ax = plt.subplots(1, 2, figsize = (20, 5))
sns.boxplot(data = default, x = 'default', y = 'balance', ax = ax[0])
sns.boxplot(data = default, x = 'default', y = 'income', ax = ax[1])

### Considering only `balance` as the predictor



In [ ]:
#create binary default column
default['binary_default'] = np.where(default['default'] == 'No', 0, 1)

In [ ]:
#scatter of Balance vs Default
sns.scatterplot(data = default, x = 'balance', y = 'binary_default')

##### PROBLEM

1. Build a `LinearRegression` model with balance as the predictor.
2. Predict the default for balances: `[200, 1000, 1500, 2000, 2500, 3500]`.  Do these make sense?

In [ ]:
X = default[['balance']]
y = default['binary_default']

In [ ]:
#instantiate and fit the model


In [ ]:
new_X = np.array([200, 1000, 1500, 2000, 2500, 3500])

In [ ]:
predictions = ''

In [ ]:
#regplot
sns.regplot(x = X, y = y)

### The Sigmoid aka Logistic Function


$$y = \frac{1}{1 + e^{-x}}$$

In [ ]:
#define the logistic
def logistic(x): return 1/(1 + np.exp(-x))

In [ ]:
#domain
x = np.arange(-10, 10, .1)

In [ ]:
#plot it
plt.plot(x, logistic(x))

### Usage should seem familiar

Fit a `LogisticRegression` estimator from `sklearn` on the features:

```python 
X = default[['balance']]
y = default['binary_default']
```

In [ ]:
#instantiate
clf = LogisticRegression()

In [ ]:
#define X and y
X = default[['balance']]
y = default['default']

In [ ]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 22)

In [ ]:
#fit on the train
clf.fit(X_train, y_train)

In [ ]:
#examine train and test scores
print(f'Train Score: {clf.score(X_train, y_train)}')
print(f'Test Score: {clf.score(X_test, y_test)}')

### Evaluating the Classifier

In `scikitlearn` the primary default evalution metric is **accuracy** or percent correct.  We still need to compare this to our baseline -- typically predicting the most frequently occurring class.  Further, you can investigate the mistakes made with each class by looking at the **Confusion Matrix**.  A quick visualization of this is had using the `ConfusionMatrixDisplay`.

In [ ]:
#baseline -- most frequently occurring class
y_train.value_counts(normalize = True)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
#from estimator
ConfusionMatrixDisplay.from_estimator(clf, X_test, y_test)

### Similarities to our earlier work

In [ ]:
#there is a coefficient
clf.coef_

In [ ]:
#there is an intercept
clf.intercept_

### Where was the line?

The version of the logistic we have just developed is actually:

$$ y = \frac{e^{ax + b}}{1 + e^{ax + b}} $$

Its output represents probabilities of being labeled the positive class in our example.  This means that we can interpret the output of the above function using our parameters, remembering that we used the `balance` feature to predict `default`.

In [ ]:
def predictor(x):
    line = clf.coef_[0]*x + clf.intercept_
    return np.e**line/(1 + np.e**line)

In [ ]:
#predict 1000
predictor(1000)

In [ ]:
#predict 2000
predictor(2000)

In [ ]:
#estimator has this too
clf.predict_proba(np.array([[1000]]))

In [ ]:
clf.predict(np.array([[1000]]))

### Using Categorical Features

In [ ]:
default.head(2)

In [ ]:
default['student_binary'] = np.where(default.student == 'No', 0, 1)

In [ ]:
X = default[['student_binary']]

In [ ]:
#instantiate and fit
clf = LogisticRegression()
clf.fit(X, y)

In [ ]:
#performance
clf.score(X, y)

In [ ]:
#coefficients
clf.coef_

In [ ]:
#compare probabilities
clf.predict_proba(X)

### Using Multiple Features



In [ ]:
default.columns

In [ ]:
features = ['balance', 'income', 'student_binary']
X = default.loc[:, features]
y = default['binary_default']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=22)

In [ ]:
clf = LogisticRegression().fit(X_train, y_train)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

**Predictions**:

- student: yes
- balance: 1,500 dollars
- income: 40,000 dollars

In [ ]:
ex1 = np.array([[1500, 40_000, 1]])
#predict probability
clf.predict_proba(ex1)

- student: no
- balance: 1,500 dollars
- income: 40,000 dollars

In [ ]:
ex2 = np.array([[1500, 40_000, 0]])
#predict probability
clf.predict_proba(ex2)

### This is similar to our multicollinearity in regression; we will call it confounding

<center>
<img src = 'https://github.com/jfkoehler/nyu_bootcamp_fa24/blob/main/images/default_confound.png?raw=true' />
</center>

#### Using `scikitlearn` and its `Pipeline`

From the original data, to build a model involved:

1. One hot or dummy encoding the categorical feature.
2. Standard Scaling the continuous features
3. Building Logistic model

we can accomplish this all with the `Pipeline`, where the first step is a `make_column_transformer` and the second is a `LogisticRegression`.  

In [ ]:
from sklearn.pipeline import Pipeline 
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(default[['student', 'income', 'balance']], default['default'],
                                                   random_state = 22)

In [ ]:
# create OneHotEncoder instance


In [ ]:
# create StandardScaler instance


In [107]:
# make column transformer -- one hot encode student, scale remainder
# transformer = make_column_transformer((), remainder = '')

In [ ]:
# logistic regressor


In [103]:
# pipeline
pipe = Pipeline([(), ()])

In [ ]:
# fit it


In [ ]:
# score on train and test
print(f'Train Score: {pipe.score(X_train, y_train)}')
print(f'Test Score: {pipe.score(X_test, y_test)}')

In [ ]:
pipe.named_steps['model'].coef_

#### Problem

Below, a dataset on bank customer churn is loaded and displayed.  Your objective is to predict `Exited` or not.  Use `CreditScore`, `Gender`, `Age`, `Tenure`, and `Balance` as predictors.  Examine the confusion matrix display.  Was your classifier better at predicting exits or non-exits?

In [ ]:
from sklearn.datasets import fetch_openml

In [ ]:
bank_churn = fetch_openml(data_id = 43390).frame

In [ ]:
bank_churn.head()

In [ ]:
#create train/test split -- random_state = 42
X = bank_churn[['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance']]
y = bank_churn['Exited']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=22)

In [ ]:
ohe = OneHotEncoder(drop = 'first')
sscaler = StandardScaler()

In [ ]:
#encode gender and standard scale other features


In [ ]:
#set up pipeline to encode/scale and then build logistic regression model


In [ ]:
#fit the model on training data


In [ ]:
#score on test and train


In [ ]:
#compare to baseline


In [ ]:
#confusion matrix with test data


#### Compare to KNN and Grid Searching

Let's compare how this estimator performs compared to the `KNeighborsClassifier`.  This time however, we will be trying many KNN models across different numbers of neighbors.  One way we could do this is with a loop; something like:

```python
for neighbor in range(1, 30, 2):
    knn = KNeighborsClassifier(n_neighbors = neighbor).fit(X_train, y_train)
```



Instead, we can use the `GridSearchCV` object from sklearn.  This will take an estimator and a dictionary with parameters to be searched over.  

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# parameters we want to try
params = {'n_neighbors': range(1, 30, 2)}

In [ ]:
# estimator with parameters


In [ ]:
# grid search object


In [ ]:
# fit it
X = default[['student_binary', 'income', 'balance']]
y = default['default']


In [ ]:
# what was best?


In [ ]:
# score it 


#### Comparing Results

A good way to think about classifier performance is using a **confusion matrix**.  Below, we visualize this using the `ConfusionMatrixDisplay.from_estimator`. 

![](https://www.sharpsightlabs.com/wp-content/uploads/2023/09/sklearn-confusion-matrix_visual-example.png)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
# a single confusion matrix
ConfusionMatrixDisplay.from_estimator(pipe, X_test, y_test, display_labels=['No', 'Yes'])

In [ ]:
# compare knn and logistic
fig, ax = plt.subplots(1, 2, figsize = (19, 5))
ConfusionMatrixDisplay.from_estimator(pipe, X_test, y_test, display_labels=['no', 'yes'], ax = ax[0])
ax[0].set_title('Logistic')
ConfusionMatrixDisplay.from_estimator(grid, X_test, y_test, display_labels=['no', 'yes'], ax = ax[1])
ax[1].set_title('KNN')

#### Activity: Stock Market Data

This data set consists of percentage returns for the S&P 500 stock index over 1,250 days, from the beginning of 2001 until the end of 2005. For each date, we have recorded the percentage returns for each of the five previous trading days, `Lag1` through `Lag5`. We have also recorded `Volume` (the number of shares traded on the previous day, in billions), `Today` (the percentage return on the date in question) and `Direction` (whether the market was Up or Down on this date). Our goal is to predict `Direction` (a qualitative response) using the other features.

In [ ]:
stocks = pd.read_csv('https://raw.githubusercontent.com/JWarmenhoven/ISLR-python/refs/heads/master/Notebooks/Data/Smarket.csv', index_col=0)

In [ ]:
stocks.head()

1. Predict `Direction` using `Lag1` through `Lag5` and `Volume` using a logistic regression model.

2. Predict `Direction` using only `Lag1` and `Lag2` as predictors.  Is this model better or worse than your first model?

3. As we don't have any test data to evaluate our estimator with, create a test set of just observations from 2005.  Fit your model on the training data and compare the train vs test performance.

4. Is this a model that you would feel comfortable using to trade with?  Why or why not?

5. Compare a KNN model performance on the test set that you have grid searched.  Is this model better or worse than your logistic regression model?  Why?

#### Practice

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
cancer = load_breast_cancer(as_frame=True).frame

In [ ]:
cancer.head(3)

In [108]:
cancer['target'] = np.where(cancer['target'] == 0, 1, 0) #ONLY RUN ONCE!  RELOAD DATA IF YOU RUN TWICE.

In [ ]:
# use all features


In [ ]:
# train/test split -- random_state = 22


In [ ]:
# pipeline to scale then logistic regressor


In [ ]:
# fit logreg pipeline


In [ ]:
# examine confusion matrix on test data


In [ ]:
# do you care about one mistake more than the other here?? Which kind?

**EXIT TICKET**

Examine the discussion from Wikipedia's page on [Precision and Recall](https://en.wikipedia.org/wiki/Precision_and_recall).  Use the confusion matrices above to compute the precision and recall of your KNN and Logistic Models.  Answer the questions linked in the form [here](https://docs.google.com/forms/d/e/1FAIpQLSes81-ynE-JkGcltSb09MW9SfvZbh3hL3hh7FDT5yI2RChwXg/viewform?usp=sharing).